In [ ]:
import numpy as np


In [ ]:
datain = np.load('initial_inputs.npy')
dataout = np.load('initial_outputs.npy')

In [ ]:
print(datain)
print(datain.shape)   # Useful if it’s an array
print(type(datain)) 

In [ ]:
print(dataout)
print(dataout.shape)   # Useful if it’s an array
print(type(dataout)) 

# Week 1

In [ ]:
from scipy.optimize import minimize
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern

X_init = datain
y_init = dataout

# Fit GP
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
gp.fit(X_init, y_init)

# Surrogate to maximise (negative for minimize)
def surrogate_neg(x):
    return -gp.predict(x.reshape(1, -1))[0]

# Bounds for normalized inputs
bounds = [(0,1), (0,1), (0,1), (0,1)]

# Try multiple random starts to avoid local issues
best_x = None
best_val = float('inf')
for _ in range(10):
    x0 = np.random.rand(4)
    res = minimize(surrogate_neg, x0=x0, bounds=bounds, method='L-BFGS-B')
    if res.fun < best_val:
        best_val = res.fun
        best_x = res.x

x_next = best_x
print("Next point to evaluate:", x_next)


# Week 2

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern

# --- Existing data ---
X_init = datain           # shape (n_samples, 4)
y_init = dataout          # shape (n_samples,)

# --- New evaluated point ---
x_new = np.array([0.973386, 0.889905, 0.981563, 0.242055])
y_new = 2755.4496930419423

# --- Add new data to dataset ---
X_all = np.vstack([X_init, x_new])
y_all = np.hstack([y_init, y_new])

# --- Refit Gaussian Process ---
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
gp.fit(X_all, y_all)

# --- Generate candidate next points around current best ---
best_x = x_new
sigma = 0.02  # tweak size for local exploration
num_candidates = 5

x_next_candidates = best_x + np.random.normal(0, sigma, size=(num_candidates, 4))
# Ensure all points are within [0,1]
x_next_candidates = np.clip(x_next_candidates, 0, 1)

print("Candidate next points to evaluate:")
print(x_next_candidates)


# Week 3

In [ ]:
import numpy as np
from scipy.optimize import minimize
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
datain = np.load('initial_inputs.npy')
dataout = np.load('initial_outputs.npy')

The new point still produced a very high yield, confirming that:
You’re indeed near the global optimum.
The response surface around the best point is smooth and relatively flat, so small parameter variations still yield high outputs.
The optimum likely lies somewhere between those two points.

✅ Adding both new high-yield points
✅ Re-fitting the Gaussian Process with improved stability
✅ Using a UCB-style acquisition for balanced exploration/exploitation
✅ Local refinement with smaller perturbations (sigma = 0.01)

In [ ]:
import numpy as np
from scipy.optimize import minimize
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern

# --- Existing data ---
X_init = datain           # shape (n_samples, 4)
y_init = dataout          # shape (n_samples,)

# --- Add the two new data points ---
new_points = np.array([
    [0.973386, 0.889905, 0.981563, 0.242055],  # best from previous run
    [0.963910, 0.868445, 0.987031, 0.295817]   # new high-yield point
])
new_outputs = np.array([
    2755.4496930419423,
    2574.150115501027
])

# Combine all data
X_all = np.vstack([X_init, new_points])
y_all = np.hstack([y_init, new_outputs])

# --- Fit updated Gaussian Process ---
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(
    kernel=kernel,
    alpha=1e-8,                # smaller noise term for precision
    normalize_y=True,
    n_restarts_optimizer=5     # more robust kernel fitting
)
gp.fit(X_all, y_all)

# --- Define acquisition function (UCB variant) ---
def surrogate_neg_ucb(x, kappa=2.0):
    mean, std = gp.predict(x.reshape(1, -1), return_std=True)
    return -(mean + kappa * std)

# --- Search bounds for normalized inputs ---
bounds = [(0, 1), (0, 1), (0, 1), (0, 1)]

# --- Optimize acquisition function for next sampling point ---
best_x, best_val = None, float('inf')
for _ in range(10):
    x0 = np.random.rand(4)
    res = minimize(surrogate_neg_ucb, x0=x0, bounds=bounds, method='L-BFGS-B')
    if res.fun < best_val:
        best_val = res.fun
        best_x = res.x

x_next = best_x

print("Suggested next point to evaluate:", x_next)

# --- Optionally: generate a few local perturbations for fine exploration ---
sigma = 0.01
num_candidates = 5
x_next_candidates = x_next + np.random.normal(0, sigma, size=(num_candidates, 4))
x_next_candidates = np.clip(x_next_candidates, 0, 1)

print("\nLocal candidate points for fine-tuning:")
print(x_next_candidates)


In [ ]:
x_next_6dp = np.round(x_next, 6)
x_next_6dp
print("Suggested next point to evaluate:", x_next_6dp)


# Week 4

In [ ]:
# New best: [0.999999, 0.999999, 0.999999, 0.024637]
# 4440.50, which is a big jump over the previous best (~2755).
# Implication 1: The optimum appears to lie at or extremely near the upper boundary for the first three variables (≈1.0).
# Implication 2: The fourth variable is near zero (≈0.0246). Earlier high points had the fourth at ~0.24–0.30;
# the new much better value suggests the peak may be at an extreme combination: first three maximized, fourth minimized.
# Implication 3: Because the best is on (or very near) the boundary, we must be careful:
# local symmetric perturbations may be misleading since you can’t go past 1.0.
# The function may be unimodal but with its maximum on the boundary.

Because the optimum appears at extremes
be cautious interpreting the GP far from observed data but your new top point strongly suggests a boundary optimum.
The single most informative next experiment is the 1D sweep on the 4th variable while holding the first three at 1.0.
That will tell you whether to push the fourth to exactly zero, or to some small positive value.

In [ ]:
import numpy as np
from scipy.optimize import minimize
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern

datain = np.load('initial_inputs.npy')
dataout = np.load('initial_outputs.npy')

# --- Existing base data ---
X_init = datain.copy()       # shape (n_samples, 4)
y_init = dataout.copy()      # shape (n_samples,)

# --- Add all known evaluated high-yield points ---
new_points = np.array([
    [0.973386, 0.889905, 0.981563, 0.242055],  # first improvement
    [0.963910, 0.868445, 0.987031, 0.295817],  # second
    [0.999999, 0.999999, 0.999999, 0.024637]   # current best
])
new_outputs = np.array([
    2755.4496930419423,
    2574.150115501027,
    4440.500188145975
])

# Combine all data
X_all = np.vstack([X_init, new_points])
y_all = np.hstack([y_init, new_outputs])

# --- Fit updated Gaussian Process ---
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(
    kernel=kernel,
    alpha=1e-8,                 # low noise for precision near smooth peak
    normalize_y=True,
    n_restarts_optimizer=8      # more robust hyperparameter fitting
)
gp.fit(X_all, y_all)

# --- Define acquisition function (UCB variant) ---
def acq_neg_ucb(x, kappa=2.0):
    x = np.asarray(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    return -(mu + kappa * sigma)

bounds = [(0,1),(0,1),(0,1),(0,1)]

# --- 1) 1D sweep over the 4th variable with first 3 fixed at 1.0 ---
fourth_grid = np.array([0.0, 0.005, 0.01, 0.02, 0.05, 0.1, 0.2, 0.3])
sweep_points = np.column_stack([
    np.ones_like(fourth_grid),
    np.ones_like(fourth_grid),
    np.ones_like(fourth_grid),
    fourth_grid
])
means = gp.predict(sweep_points)
print("1D sweep (first 3 = 1.0) — predicted mean yields:")
for x, m in zip(sweep_points, means):
    print(x, "→", m)

top_idx = np.argsort(means)[-3:][::-1]
top_sweep_points = sweep_points[top_idx]

# --- 2) Acquisition optimization with boundary-aware seeds ---
best_x, best_val = None, float('inf')
seeds = [
    new_points[-1],                          # current best
    np.array([1.0, 1.0, 1.0, 0.0]),
    np.array([0.995, 0.995, 0.995, 0.01]),
    np.array([1.0, 1.0, 1.0, 0.05]),
]
for _ in range(8):
    jitter = np.random.normal(0, 0.01, 4)
    seeds.append(np.clip(new_points[-1] + jitter, 0, 1))

for x0 in seeds:
    res = minimize(acq_neg_ucb, x0=x0, bounds=bounds, method='L-BFGS-B',
                   options={'maxiter': 200})
    if res.fun < best_val:
        best_val = res.fun
        best_x = res.x

print("\nAcquisition-opt suggested next point:", best_x, "acq value:", -best_val)

# --- 3) Local fine candidates around best_x ---
sigma = 0.005
num_candidates = 6
local_candidates = np.clip(
    best_x + np.random.normal(0, sigma, size=(num_candidates, 4)), 0, 1
)

# Combine with top sweep points for next tests
candidates = np.vstack([top_sweep_points, local_candidates, best_x.reshape(1, -1)])

print("\nCandidate next points to evaluate:")
for i, c in enumerate(candidates):
    print(f"{i+1}: {c}")


In [ ]:
import numpy as np
from scipy.optimize import minimize
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern

datain = np.load('initial_inputs.npy')
dataout = np.load('initial_outputs.npy')

# --- Existing base data ---
X_init = datain.copy()       # shape (n_samples, 4)
y_init = dataout.copy()      # shape (n_samples,)

# --- Add all known evaluated high-yield points ---
new_points = np.array([
    [0.973386, 0.889905, 0.981563, 0.242055],  # first improvement
    [0.963910, 0.868445, 0.987031, 0.295817],  # second
    [0.999999, 0.999999, 0.999999, 0.024637]   # current best
])
new_outputs = np.array([
    2755.4496930419423,
    2574.150115501027,
    4440.500188145975
])

# --- Combine all data ---
X_all = np.vstack([X_init, new_points])
y_all = np.hstack([y_init, new_outputs])

# --- Fit updated Gaussian Process ---
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(
    kernel=kernel,
    alpha=1e-8,                 # low noise for precision near smooth peak
    normalize_y=True,
    n_restarts_optimizer=8      # robust hyperparameter fitting
)
gp.fit(X_all, y_all)

# --- Define acquisition function (UCB variant) ---
def acq_neg_ucb(x, kappa=2.0):
    x = np.asarray(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    return -(mu + kappa * sigma)

# --- TIGHTENED BOUNDS to avoid 0.0 or 1.0 ---
bounds = [(0.02, 0.98)] * 4

# --- 1) 1D sweep over the 4th variable with first 3 fixed at 0.98 (not 1.0 anymore) ---
fourth_grid = np.array([0.02, 0.03, 0.05, 0.1, 0.2, 0.3])
sweep_points = np.column_stack([
    np.full_like(fourth_grid, 0.98),
    np.full_like(fourth_grid, 0.98),
    np.full_like(fourth_grid, 0.98),
    fourth_grid
])
means = gp.predict(sweep_points)
print("1D sweep (first 3 = 0.98) — predicted mean yields:")
for x, m in zip(sweep_points, means):
    print(x, "→", m)

top_idx = np.argsort(means)[-3:][::-1]
top_sweep_points = sweep_points[top_idx]

# --- 2) Acquisition optimization with boundary-aware seeds ---
best_x, best_val = None, float('inf')
seeds = [
    new_points[-1],                          # current best
    np.array([0.98, 0.98, 0.98, 0.02]),
    np.array([0.97, 0.97, 0.97, 0.05]),
    np.array([0.98, 0.98, 0.98, 0.1]),
]
for _ in range(8):
    jitter = np.random.normal(0, 0.01, 4)
    seeds.append(np.clip(new_points[-1] + jitter, 0.02, 0.98))

for x0 in seeds:
    res = minimize(acq_neg_ucb, x0=x0, bounds=bounds, method='L-BFGS-B',
                   options={'maxiter': 200})
    if res.fun < best_val:
        best_val = res.fun
        best_x = res.x

print("\nAcquisition-opt suggested next point:", best_x, "acq value:", -best_val)

# --- 3) Local fine candidates around best_x ---
sigma = 0.005
num_candidates = 6
local_candidates = np.clip(
    best_x + np.random.normal(0, sigma, size=(num_candidates, 4)), 0.02, 0.98
)

# Combine with top sweep points for next tests
candidates = np.vstack([top_sweep_points, local_candidates, best_x.reshape(1, -1)])

print("\nCandidate next points to evaluate:")
for i, c in enumerate(candidates):
    print(f"{i+1}: {c}")


# Week 5
- New point: [0.98, 0.98, 0.98, 0.02] → 3669.14.
- Compare with prior highs:
[0.9639, 0.8684, 0.9870, 0.2958] → 2574
[0.9734, 0.8899, 0.9816, 0.2421] → 2755
[0.999999,0.999999,0.999999,0.024637] → 4440.50 (best so far)
- the model appears monotonic (or strongly increasing) in x1–x3 toward their upper edge, and prefers a small positive x4 near ~0.02–0.03.

In [ ]:
import numpy as np
from scipy.optimize import minimize
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern

datain = np.load('initial_inputs.npy')
dataout = np.load('initial_outputs.npy')

# --- Existing base data ---
X_init = datain.copy()       # shape (n_samples, 4)
y_init = dataout.copy()      # shape (n_samples,)

# --- Add all known evaluated high-yield points ---
new_points = np.array([
    [0.973386, 0.889905, 0.981563, 0.242055],  # first improvement
    [0.963910, 0.868445, 0.987031, 0.295817],  # second
    [0.999999, 0.999999, 0.999999, 0.024637],  # current best
    [0.98, 0.98, 0.98, 0.02] 
])
new_outputs = np.array([
    2755.4496930419423,
    2574.150115501027,
    4440.500188145975,
    3669.139092257369
])

# --- Combine all data ---
X_all = np.vstack([X_init, new_points])
y_all = np.hstack([y_init, new_outputs])

In [ ]:
# assume datain / dataout already loaded as before
import numpy as np
from scipy.optimize import minimize
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern

# fit GP (same settings)
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-8, normalize_y=True, n_restarts_optimizer=8)
gp.fit(X_all, y_all)

# bounds (tightened)
bounds = [(0.02, 0.98)] * 4

# 1D sweep for x4 with first three = 0.98
fourth_grid = np.array([0.02, 0.025, 0.03, 0.04, 0.06])
sweep_points = np.column_stack([np.full_like(fourth_grid, 0.98),
                                np.full_like(fourth_grid, 0.98),
                                np.full_like(fourth_grid, 0.98),
                                fourth_grid])
sweep_means = gp.predict(sweep_points)
for p, m in zip(sweep_points, sweep_means):
    print(p, "→ predicted mean:", m)

# choose best sweep x4 (predicted) and create sensitivity tests for x1-3
best_idx = np.argmax(sweep_means)
best_x4 = fourth_grid[best_idx]

first_three_tests = np.array([[v, v, v, best_x4] for v in [0.95, 0.97, 0.98]])
print("\nFirst-3 sensitivity candidates (x4 fixed):")
for p in first_three_tests:
    print(p, "→", gp.predict(p.reshape(1,-1))[0])

# local perturbations around [0.98,0.98,0.98,best_x4]
sigma = 0.005
local_cands = np.clip(np.tile([0.98,0.98,0.98,best_x4], (6,1)) + np.random.normal(0, sigma, (6,4)), 0.02, 0.98)
print("\nLocal perturbation candidates:")
for p in local_cands:
    print(p, "→", gp.predict(p.reshape(1,-1))[0])


In [ ]:
# --- Assumes gp is already fitted and new_points etc. are in place ---
import numpy as np
from scipy.optimize import minimize

# 1) Reuse the previously computed best_x if you have it; otherwise run acquisition optimization quickly:
def acq_neg_ucb(x, kappa=2.0):
    x = np.asarray(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    return -(mu + kappa * sigma)

# quick acquisition optimization to get a candidate (optional)
bounds = [(0.02,0.98)]*4
best_x = None
best_val = float('inf')
seeds = [new_points[-1], np.array([0.98,0.98,0.98,0.02]), np.array([0.97,0.97,0.97,0.05])]
for _ in range(6):
    jitter = np.random.normal(0, 0.01, 4)
    seeds.append(np.clip(new_points[-1] + jitter, 0.02, 0.98))

for x0 in seeds:
    res = minimize(acq_neg_ucb, x0=x0, bounds=bounds, method='L-BFGS-B', options={'maxiter':100})
    if res.fun < best_val:
        best_val = res.fun
        best_x = res.x

# 2) Build candidate list: sweep on x4 (x1-3 = 0.98), the acquisition best_x, and a few local perturbs
fourth_grid = np.array([0.02, 0.025, 0.03, 0.04, 0.06])
sweep_points = np.column_stack([
    np.full_like(fourth_grid, 0.98),
    np.full_like(fourth_grid, 0.98),
    np.full_like(fourth_grid, 0.98),
    fourth_grid
])

# local perturbations around best_x
np.random.seed(0)
local = np.clip(best_x + np.random.normal(0, 0.005, size=(5,4)), 0.02, 0.98)

# combine and deduplicate
candidates = np.vstack([sweep_points, local, best_x.reshape(1,-1)])
# optional: remove duplicates (numerically)
unique_candidates = np.unique(np.round(candidates, 6), axis=0)

# 3) Evaluate GP predictive mean and pick single best
means = gp.predict(unique_candidates)
best_idx = np.argmax(means)
single_best = unique_candidates[best_idx]

print("Single best candidate (highest predicted mean):", single_best)
print("Predicted mean yield:", means[best_idx])


In [ ]:
x_next_6dp = np.round(single_best, 6)
print("Rounded to:", x_next_6dp)

# Week 6
- [0.980000, 0.980000, 0.980000, 0.060000] Essentially identical to previous point (~0.0001 higher)
- [0.999999,0.999999,0.999999,0.024637] 4440 Peak observed so far, very high first three inputs, small x4 (~0.0246)

- By keeping bounds artificially tight I may be preventing access to higher yields.
- Relaxing the bounds toward 1.0 will very likely increase yield

In [ ]:
import numpy as np
from scipy.optimize import minimize
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern

datain = np.load('initial_inputs.npy')
dataout = np.load('initial_outputs.npy')

# --- Existing base data ---
X_init = datain.copy()       # shape (n_samples, 4)
y_init = dataout.copy()      # shape (n_samples,)

# --- Add all known evaluated high-yield points ---
new_points = np.array([
    [0.973386, 0.889905, 0.981563, 0.242055],  # first improvement
    [0.963910, 0.868445, 0.987031, 0.295817],  # second
    [0.999999, 0.999999, 0.999999, 0.024637],  # current best
    [0.98, 0.98, 0.98, 0.02],
    [0.980000, 0.980000, 0.980000, 0.060000]
])
new_outputs = np.array([
    2755.4496930419423,
    2574.150115501027,
    4440.500188145975,
    3669.139092257369,
    3669.260114942143
])

# --- Combine all data ---
X_all = np.vstack([X_init, new_points])
y_all = np.hstack([y_init, new_outputs])

In [ ]:
import numpy as np
from scipy.optimize import minimize
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern

# --- LOAD EXISTING DATA (your above snippet) ---

# --- Fit GP (same settings you've been using) ---
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-8, normalize_y=True, n_restarts_optimizer=8)
gp.fit(X_all, y_all)

# --- Build candidate set that probes toward the boundary safely ---
# grid for x1-x3 near boundary (you can change values)
x_levels = np.array([0.98, 0.985, 0.99, 0.995])  # include 1.0 only if allowed
# keep x4 near the small region we've seen best
x4_levels = np.array([0.02, 0.025, 0.03, 0.04])

candidates = []
for a in x_levels:
    for b in x4_levels:
        candidates.append([a, a, a, b])
candidates = np.array(candidates)

# Optionally, remove candidates equal to points you've already tested (to avoid duplication)
# (simple numeric comparison, adjust tolerance if necessary)
def not_duplicate(c, X_existing, tol=1e-6):
    return not np.any(np.all(np.abs(X_existing - c) <= tol, axis=1))

filtered = np.array([c for c in candidates if not_duplicate(c, X_all)])
if filtered.shape[0] == 0:
    filtered = candidates  # fallback if everything was present

# --- Predictive means and uncertainties ---
means, stds = gp.predict(filtered, return_std=True)

# --- Rank by predicted mean (descending) ---
order = np.argsort(means)[::-1]
ranked = filtered[order]
ranked_means = means[order]
ranked_stds = stds[order]

# Print top-k candidates
k = min(6, len(ranked))
print("Top candidates to probe toward boundary (ranked by GP mean):")
for i in range(k):
    print(f"{i+1}: {ranked[i]}  predicted_mean={ranked_means[i]:.2f}, std={ranked_stds[i]:.2f}")
